In [4]:
from pycoingecko import CoinGeckoAPI
import numpy as np
import sqlite3
from tqdm import tqdm
import datetime 
import time
from urllib.error import HTTPError
import random
import pandas as pd

In [2]:
cg = CoinGeckoAPI()

### Get coin data

In [ ]:
coins_markets = []
for page in tqdm(range(100)):
    results = cg.get_coins_markets(vs_currency='usd', per_page=250, page=page, order='id_asc')
    if len(results) > 0:
        coins_markets += results
    else:
        break

### Insert to Coins

In [ ]:
conn = sqlite3.connect('crypto.sqlite3')
cursor = conn.cursor()

for coin in coins_markets:
    id = coin['id']
    symbol = coin['symbol']
    name = coin['name']
    current_price = coin['current_price']
    market_cap = coin['market_cap']
    total_volume = coin['total_volume']
    circulating_supply = coin['circulating_supply']
    last_updated = coin['last_updated']
    try:
        insert_coin_query = f"""INSERT INTO Coins(id, name, symbol, price, volume, marketcap, circulatingsupply, date) 
                VALUES ('{id}','{name}','{symbol}',{current_price},{total_volume}, {market_cap}, {circulating_supply}, '{last_updated.split('T')[0]}')"""
        cursor.execute(insert_coin_query)
    except:
        pass

conn.commit()
cursor.close()

### Get coin id in database

In [33]:
conn = sqlite3.connect('crypto.sqlite3')
cursor = conn.cursor()
insert_coin_query = "SELECT id from Coins"
coins_id = cursor.execute(insert_coin_query)
coins_id = coins_id.fetchall()
cursor.close()
conn.close()

### Get history data

In [ ]:
i = 0
history = []
coins = []

In [ ]:
while True:
    try:
        for coin in coins_id[i:]:
            try:
                coin_market_chart = cg.get_coin_market_chart_by_id(coin[0], vs_currency='usd', days='max')
            except ValueError:
                pass

            prices = np.array(coin_market_chart['prices'])[:, 1][::-1]
            market_caps = np.array(coin_market_chart['market_caps'])[:, 1][::-1]
            total_volumes = np.array(coin_market_chart['total_volumes'])[:, 1][::-1]
            history.append((prices, total_volumes, market_caps))
            coins.append(coin)
            if i % 100 == 0:
                print(i)
            i += 1
    except Exception:
        print(len(history))
        time.sleep(10)

### Insert to CoinHistory

In [ ]:
conn = sqlite3.connect('crypto.sqlite3')
cursor = conn.cursor()

one_day = datetime.timedelta(days=1)
for (prices, total_volumes, market_caps), coin_id in zip(history, coins):
    date = datetime.datetime.now()
    for price, volume, marketcap in zip(prices, total_volumes, market_caps):
        try:
            insert_coin_query = f"""INSERT INTO CoinHistory(id, date, price, volume, marketcap) 
                        VALUES ('{coin_id[0]}','{date.date()}',{price},{volume},{marketcap})"""
            cursor.execute(insert_coin_query)
            date = date - one_day
        except:
            pass

conn.commit()
cursor.close()

### Random Transactions

1. ID: Auto
2. Category ID:
    - 1: Trade
    - [2, 9]: Incoming
    - [10, 16]: Outgoing
3. InCoinID, OutCoinID, FeeCoinID: coins_id
4. InQty, OutQty: 0.01 -> 10000
5. FeeQty: 0.01 -> 10
6. Date: 1/1/2021 -> 1/1/2022

In [141]:
def random_date():
    start = datetime.datetime(2021, 1, 1)
    end = datetime.datetime(2022, 1, 1)
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    dt = start + datetime.timedelta(seconds=random_second)
    return dt.date()

In [245]:
conn = sqlite3.connect('crypto.sqlite3')
cursor = conn.cursor()

for i in range(100000):
    category_id = random.randint(1, 16)
    date = random_date()
    InCoinID = random.choice(coins_id)[0]
    OutCoinID = random.choice(coins_id)[0]
    FeeCoinID = InCoinID
    InQty = random.random() * random.randint(1, 10000)
    OutQty = random.random() * random.randint(1, 10000)
    FeeQty = random.random() * InQty / 100
    if category_id == 1:
        insert_coin_query = f"""INSERT INTO Transactions(categoryid, date, incoinid, inqty, outcoinid, outqty, feecoinid, feeqty) 
                        VALUES ({category_id},'{date}','{InCoinID}',{InQty},'{OutCoinID}',{OutQty},'{FeeCoinID}',{FeeQty})"""
    elif category_id >= 2 and category_id <= 9:
        insert_coin_query = f"""INSERT INTO Transactions(categoryid, date, incoinid, inqty, feecoinid, feeqty) 
                        VALUES ({category_id},'{date}','{InCoinID}',{InQty},'{FeeCoinID}',{FeeQty})"""
    else:
        insert_coin_query = f"""INSERT INTO Transactions(categoryid, date, outcoinid, outqty, feecoinid, feeqty) 
                        VALUES ({category_id},'{date}','{OutCoinID}',{OutQty},'{FeeCoinID}',{FeeQty})"""
    cursor.execute(insert_coin_query)
    
conn.commit()
cursor.close()

### Random Futures

1. ID: Auto
2. Coin ID:
3. Mode: [Isolated, Cross]
4. Type: [LONG, SHORT]
5. RR: [1 -> 8]
6. Leverage: [1, 125]
7. Margin: 1 USD -> 100000 USD
8. PNL: Margin * RR * (1, -1)
9. Fee: Margin * 0.0005
10. Date: 1-1-2021 -> 1-1-2022

In [246]:
def random_date():
    start = datetime.datetime(2021, 1, 1)
    end = datetime.datetime(2022, 1, 1)
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = random.randrange(int_delta)
    dt = start + datetime.timedelta(seconds=random_second)
    return dt.date()

In [454]:
conn = sqlite3.connect('crypto.sqlite3')
cursor = conn.cursor()

for i in range(100000):
    coinID = random.choice(coins_id)[0]
    mode = random.choice(['isolated', 'cross'])
    _type = random.choice(['long', 'short'])
    RR = random.random() * random.randint(1, 10)
    leverage = random.randint(1, 125)
    margin = random.randint(1, 100000) # USDT
    PNL = random.random() * margin * RR # USDT
    fee = margin * 0.0005 # USDT
    date = random_date()

    insert_coin_query = f"""INSERT INTO Futures(coinid, mode, type, rr, leverage, margin, pnl, fee, date) 
                    VALUES ('{coinID}','{mode}','{_type}',{RR},{leverage},{margin},{PNL},{fee}, '{date}')"""
    cursor.execute(insert_coin_query)

conn.commit()
cursor.close()